In [14]:
# 이미지 처리 분야에서 가장 유명한 신경망 모델인 CNN 을 이용하여 더 높은 인식률을 만들어봅니다.
import tensorflow as tf
from PIL import Image
import glob
import numpy as np

image_width = 224
image_height = 224
image_label = []
All_Img = []
#label = np.zeros(3)
label = np.eye(3)[image_label]

In [15]:
image_file_list = glob.glob("./lemon/*.jpg")
for img in image_file_list:
    image_label.append(0)
    image = Image.open(img)
    All_Img.append(np.float32(image))


image_file_list = glob.glob("./hanlabong/*.jpg")
for img in image_file_list:
    image_label.append(1)
    image = Image.open(img)
    All_Img.append(np.float32(image))

image_file_list = glob.glob("./tangerine/*.jpg")
for img in image_file_list:
    image_label.append(2)
    image = Image.open(img)
    All_Img.append(np.float32(image))


In [ ]:
X = tf.placeholder(tf.float32, [None, image_width, image_height, 3])
Y = tf.placeholder(tf.float32, [None, 3])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3,3,3, 128], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3,3,128, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W3 = tf.Variable(tf.random_normal([3,3,64, 32], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W4 = tf.Variable(tf.random_normal([8*8*32, 32], stddev=0.01))
L4 = tf.reshape(L3, [-1, 8*8*32])
L4 = tf.matmul(L4, W4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.Variable(tf.random_normal([32,3], stddev=0.01))
model = tf.matmul(L4, W5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(10):
    _, cost_val = sess.run([optimizer, cost], feed_dict={X: All_Img, Y: label, keep_prob: 0.7})
    print('epoch-', epoch+1, ': ', cost_val)
print('end')

In [22]:
image_file_list = glob.glob("./lemon/*.jpg")
for img in image_file_list:
    image_label.append(0)
    image = Image.open(img)
    All_Img.append(np.float32(image))

print("Test")

predict = tf.argmax(model, 1)
original = tf.argmax(Y, 1)
is_correct = tf.equal(predict, original)

print(sess.run(predict, feed_dict={X: All_Img, keep_prob: 1}))
print(sess.run(predict, feed_dict={Y: image_label, keep_prob: 1}))

accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:'sess.run(accuracy, feed_dict={X: All_Img, Y:image_label, keep_prob:1}))

ValueError: Cannot feed value of shape (0, 3) for Tensor 'Placeholder_13:0', which has shape '(?, 2)'